1. Write a program that uses both upward and downward recursion to calculate the spherical Bessel function $j_{l}\left(  x\right)$ for the first 25 $l$ values for $x=0.1,1,10$. Tune your program so that at lease one method gives "good" values (meaning a relative error $\simeq 10^{-10}$).  
    a) Show the convergence and stability of your results.  
    b) Compare the upward and downward recursion methods, printing out $l,$ $j_{l}^{\left(  \text{up}\right)  },$ $j_{l}^{\left(  \text{down}\right)  },$ and the relative difference $\left\vert j_{l}^{\left(  \text{up}\right)
    }-j_{l}^{\left(  \text{down}\right)  }\right\vert \left/  \left(  \left\vert
    j_{l}^{\left(  \text{up}\right)  }\right\vert +\left\vert j_{l}^{\left(
    \text{down}\right)  }\right\vert \right)  \right.$.

2.  Suppose that the equations of motion for a projectile are 
    $$
    \begin{aligned}
    y&=f(t)=9600(1-e^{-t/15})-480t\\
    x&=r(t)=2400(1-e^{-t/15})
    \end{aligned}
    $$
    a) Find the elapsed time until impact accurate to 10 decimal places.

    b) Find the range accurate to 10 decimal places.

    by applying

    1) the bisection algorithm,

    2) the inverse interpolation method, and

    3) the secant method.

    Please compare your results. 
    
    **Hint**: Please read the sample scripts first.

In [ ]:
# bisection method ========================================================


3. Solve Nonlinear Equation for a Vibrating Beam  
    An important engineering problem that arises in a lot of applications is the vibrations of a clamped beam where the other end is free. This problem can be analyzed analytically, but the calculations boil down to solving the following nonlinear algebraic equation:  
    $$  
    \cosh \beta \cos \beta = −1  
    $$  
where $\beta$ is related to important beam parameters through  
$$
\beta^4 = \omega^2\frac{\rho A}{EI}
$$
where $\rho$ is the density of the beam, $A$ is the area of the cross section, $E$ is Young’s modulus, and $I$ is the moment of the inertia of the cross section. The most important parameter of interest is $\omega$, which is the frequency of the beam. We want to compute the frequencies of a vibrating steel beam with a rectangular cross section having width $b = 25$ mm and height $h = 8$ mm. The density of steel is 7850 kg/m$^3$, and $E = 2 × 10^{11}$ Pa. The moment of inertia of a rectangular cross section is $I =bh^3/12$.  

a) Plot the equation to be solved so that one can inspect where the zero crossings occur.

**Hint**:  When writing the equation as $f (\beta) = 0$, the $f$ function increases its amplitude dramatically with $\beta$. It is therefore wise to look at an equation with damped amplitude, $g(\beta) = e^{−\beta}f (\beta) = 0$. Plot $g$ instead.  

b) Compute the first three frequencies.

